In [12]:
def x_finding(state):
    N = len(state)
    for i in range (N):
        for j in range (N):
            if not state[i][j]:
                return N - i

In [13]:
def inv_function(list_row):
    N = len(list_row)
    inv_count= 0
    for i in range (N):
        for j in range (i+1, N, 1):
            if list_row[j] and list_row[i] > list_row[j] :
                inv_count += 1
    return inv_count

In [14]:
def make_1D(state):
    arr = []
    size = len(state)
    for i in range(size):
        for j in range(size):
            arr.append(state[i][j])
    return arr

In [15]:
def is_solvable(state):
    count = inv_function(make_1D(state))
    X = x_finding(state)
    N = len(state)
    if N % 2 == 1:
        return count % 2 == 0
    else:
        if count % 2 == 0:
            return X % 2 == 1
        else:
            return X % 2 == 0

In [16]:
#main function input from the file
import sys
print("Task01")
list_problem = []     
with open('problem.txt', 'r') as file:
    N = int(file.readline().strip())
    for _ in range(N):
        list_row = ([int(x) for x in file.readline().strip().split()])
        i = 0
        d = int((len(list_row))**0.5)
        state = []
        for _ in range(d):
            mat = []
            for _ in range(d):
                mat.append(list_row[i])
                i += 1
            state.append(mat)
        list_problem.append(state)
        if is_solvable(state):
            print(list_row,"\ncan be solved")
        else:
            print(list_row, "\n can't be solved")
        print("\n")


Task01
[0, 3, 1, 2] 
 can't be solved


[0, 3, 2, 1, 6, 4, 8, 7, 5] 
can be solved


[3, 14, 13, 7, 4, 5, 6, 1, 11, 12, 8, 0, 15, 9, 2, 10] 
can be solved


[4, 8, 7, 5, 0, 3, 2, 1, 6] 
can be solved


[5, 6, 1, 11, 12, 8, 0, 15, 9, 2, 10, 3, 14, 13, 7, 4] 
 can't be solved




In [17]:
#Task 2
class Node:
    def __init__(self, state=None, cost=None, hr = 0, parent=None, action=None):
            self.state= state
            self.action = action
            self.parent = parent
            self.cost = cost
            self.hr = hr
    def __lt__(self, other):
        return self.hr < other.hr
    def __gt__(self, other):
        return self.hr > other.hr

In [18]:
def swap(state, i, j, action):
    if action == 'UP':
        state[i][j] = state[i-1][j]
        state[i-1][j] = 0
    elif action == 'DOWN':
        state[i][j] = state[i+1][j]
        state[i+1][j] = 0
    elif action == 'RIGHT':
        state[i][j] = state[i][j + 1]
        state[i][j+1] = 0
    else:
        state[i][j] = state[i][j - 1]
        state[i][j-1] = 0

In [19]:
import copy as cp
class Problem:
    def __init__(self, init_state, goal):
        self.init_state = init_state
        self.goal = goal

    def goal_test(self, state):
        return state == self.goal
           
    def heuristic(self,state):
        cost = 0
        for i in range(len(state)):
            for j in range(len(state)):
                if state[i][j] == self.goal[i][j]:
                    cost += 0
                else:
                    cost +=1
        return cost
    
    def actions(self, state):
        x_pos = -1
        y_pos = -1
        state_and_actions_list = []
        size = len(state[0])
        for i in range(size):
            for j in range(size):
                if state[i][j] == 0:
                    x_pos = i
                    y_pos = j
        temp = None
        if x_pos > 0:
            temp = cp.deepcopy(state)
            swap(temp,x_pos,y_pos, 'UP')
            state_and_actions_list.append({'state':temp, 'action':'UP'})
        if x_pos + 1 < size:
            temp = cp.deepcopy(state)
            swap(temp,x_pos,y_pos, 'DOWN')
            state_and_actions_list.append({'state':temp, 'action':'DOWN'})
        if y_pos > 0:
            temp = cp.deepcopy(state)
            swap(temp,x_pos,y_pos, 'LEFT')
            state_and_actions_list.append({'state':temp, 'action':'LEFT'})
        if y_pos + 1 < size:
            temp = cp.deepcopy(state)
            swap(temp,x_pos,y_pos, 'RIGHT')
            state_and_actions_list.append({'state':temp, 'action':'RIGHT'})
        return state_and_actions_list

In [20]:
def solution(node):
    path = []
    while node.parent:
        path.append(node.action)
        node = node.parent
    path.reverse()
    return path

In [21]:
import heapq as hq
def Breath_First_Search(problem):
    state = problem.init_state
    heuristic_cost = problem.heuristic(state)
    current_Node = Node(state,0, heuristic_cost , None, None)
    frontier = []
    hq.heappush(frontier, current_Node)
    exposed = set()
    if problem.goal_test(current_Node.state):
        return solution(current_Node)
    while frontier:
        current_Node = hq.heappop(frontier)
        exposed.add(str(current_Node.state))
        if problem.goal_test(current_Node.state):
            print("Path found:")
            return solution(current_Node)
        else:
            #print("current state")
            # for i in current_Node.state:
            #     print(i)
            #print("\nchild states")
            for obj in problem.actions(current_Node.state):
                state = obj['state']
                action = obj['action']
                # print (action)
                # for i in state:
                #     print(i)
                
                heuristic_cost = problem.heuristic(state)
                child = Node(state,0, heuristic_cost, current_Node, action)
                if not str(child.state) in exposed and not child in frontier:
                   hq.heappush(frontier, child)
        #print('\n')
    return None

In [22]:
def make_2d_goal(row, col):
    element = 0
    goal = []
    for _ in range(row):
        list_row = []
        for _ in range(col):
            list_row.append(element + 1)
            element +=  1
        goal.append(list_row)
    goal[row-1][col-1]  = 0 
    return goal

#main task2
for puzzle in list_problem:
    if is_solvable(puzzle):
        #making of goal
        goal = make_2d_goal(len(puzzle), len(puzzle[0]))
        problem = Problem(puzzle, goal)
        path =  Breath_First_Search(problem)
        for action in path:
            print(action, end= "->")
        print('\n')
    input("\nPress Enter for next problem: ")

Path found:
DOWN->DOWN->RIGHT->RIGHT->UP->LEFT->DOWN->RIGHT->UP->UP->LEFT->DOWN->RIGHT->DOWN->LEFT->LEFT->UP->RIGHT->UP->LEFT->DOWN->DOWN->RIGHT->UP->LEFT->UP->RIGHT->DOWN->DOWN->RIGHT->UP->LEFT->UP->RIGHT->DOWN->DOWN->LEFT->UP->RIGHT->UP->LEFT->DOWN->LEFT->UP->RIGHT->DOWN->RIGHT->UP->LEFT->LEFT->DOWN->RIGHT->UP->RIGHT->DOWN->LEFT->LEFT->DOWN->RIGHT->RIGHT->UP->LEFT->DOWN->LEFT->UP->RIGHT->RIGHT->DOWN->

Path found:
LEFT->LEFT->LEFT->UP->RIGHT->RIGHT->DOWN->LEFT->DOWN->LEFT->UP->RIGHT->DOWN->RIGHT->RIGHT->UP->UP->UP->LEFT->DOWN->RIGHT->UP->LEFT->LEFT->DOWN->DOWN->DOWN->RIGHT->RIGHT->UP->UP->UP->LEFT->LEFT->DOWN->DOWN->DOWN->RIGHT->RIGHT->UP->UP->UP->LEFT->LEFT->DOWN->RIGHT->RIGHT->UP->LEFT->DOWN->LEFT->UP->RIGHT->RIGHT->DOWN->LEFT->UP->LEFT->DOWN->RIGHT->RIGHT->UP->LEFT->DOWN->LEFT->UP->RIGHT->DOWN->RIGHT->UP->LEFT->LEFT->DOWN->RIGHT->UP->RIGHT->DOWN->DOWN->DOWN->LEFT->UP->RIGHT->UP->LEFT->DOWN->DOWN->RIGHT->UP->UP->LEFT->UP->RIGHT->DOWN->DOWN->DOWN->LEFT->UP->UP->RIGHT->DOWN->LEFT->DO